In [1]:
using thesis, PRMaps
using Pandas
using Healpix
using Plots

import Stripeline as Sl

In [ ]:
strip = read_pickle("../instruments/lspe_strip_instrument.pkl")

In [ ]:
maps = get_foreground_maps(strip, "s0", 512);

In [ ]:
Plots.plot(maps[1].i)